See the README for an explanation of how this code runs and functions.

Contact michaeldezube at gmail dot com with questions.

In [ ]:
import table_connector

import pandas as pd
from IPython.display import display
from IPython.display import HTML

In [ ]:
fully_merged_messages_df, address_book_df = table_connector.get_cleaned_fully_merged_messages()

### Use `fully_merged_messages_df` and `address_book_df` for analysis, they contain all messages with columns for the sender and all contacts, respectively

In [ ]:
# Visualize and output a word tree.

import json
import wordtree

CONTACT_NAME = 'Mom'  # Freely change this value.
ROOT_WORD = 'feel'  # Freely change this value, note emojis are supported.

filtered_texts = fully_merged_messages_df[(fully_merged_messages_df.full_name == CONTACT_NAME)]
print 'I exchanged {0:,} texts with {1}'.format(filtered_texts.shape[0], CONTACT_NAME)

filtered_texts_i_sent = filtered_texts[filtered_texts.is_from_me == 1]
print 'I sent {0:,} texts to {1}'.format(filtered_texts_i_sent.shape[0], CONTACT_NAME)

filtered_texts_i_received = filtered_texts[filtered_texts.is_from_me == 0]
print 'I received {0:,} texts from {1}'.format(filtered_texts_i_received.shape[0], CONTACT_NAME)


# Returns JSON in the format needed for word trees.
def get_google_json_for_dataframe(df):
    array_for_json = [[text[1]] for text in df.text.iteritems()]
    array_for_json.insert(0, [['Phrases']])
    return json.dumps(array_for_json)

# You can change tree_type to 'double', 'suffix', or 'prefix', see 
# https://developers.google.com/chart/interactive/docs/gallery/wordtree#configuration-options for details
HTML(wordtree.get_word_tree_html(get_google_json_for_dataframe(filtered_texts_i_received),
                                 ROOT_WORD, lowercase=True, tree_type='double'))

In [ ]:
# Just a bit of fun to show you who you text the most.
# Note "nan" means the number was not found in your address book.

def get_message_counts(dataframe):
    return pd.Series({'Texts sent': dataframe[dataframe.is_from_me == 1].shape[0],
                      'Texts received': dataframe[dataframe.is_from_me == 0].shape[0],
                      'Texts exchanged': dataframe.shape[0]})
messages_grouped = fully_merged_messages_df.groupby('full_name').apply(get_message_counts)
messages_grouped = messages_grouped.sort_values(by='Texts exchanged', ascending=False)
messages_grouped.head(10)  # Change this value to see more rows

In [ ]:
# A bit more fun.
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

messages_grouped.head(10).plot(figsize=(20,10), kind='bar')  # Change this value to see more rows